In [ ]:
import time
import hmac
from typing import Optional, Dict, Any, List
from requests import Request, Session, Response

class ftxAPI():
    def __init__(self):
        self._api_host = 'https://ftx.com/api'
        self._api_key = b'1vyg6pC1WY-ODMhSyeHasYBV8ZVrgpAKYEu4uqWZ'
        self._api_secret  = 'qXuNtAcsQfZZEbzHlrm_rdBOtCrGgFHrYjdkshw8'
        self.ts = int(time.time() * 1000)
        self._subaccount_name = ""
        self._session = Session()
        
    def _get(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        return self._request('GET', path, params=params)
    
    def _post(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        return self._request('POST', path, params=params)
    
    def _request(self, method: str, path: str, **kwargs) -> Any:
        request = Request(method, self._api_host + path, **kwargs)
        self._sign_request(request)
        response = self._session.send(request.prepare())
        return self._process_response(response)

    def _sign_request(self, request: Request) -> None:
        self.ts = int(time.time() * 1000)
        prepared = request.prepare()
        signature_payload = f'{self.ts}{prepared.method}{prepared.path_url}'.encode()
        if prepared.body:
            signature_payload += prepared.body
        signature = hmac.new(self._api_secret.encode(), signature_payload, 'sha256').hexdigest()
        request.headers['FTX-KEY'] = self._api_key
        request.headers['FTX-SIGN'] = signature
        request.headers['FTX-TS'] = str(self.ts)
        if self._subaccount_name:
            request.headers['FTX-SUBACCOUNT'] = urllib.parse.quote(self._subaccount_name)
    
    def _process_response(self, response: Response) -> Any:
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            if not data['success']:
                raise Exception(data['error'])
            return data['result']
    
    
    def get_ticker(self,market_name):
        return self._get(f'/markets/{market_name}/orderbook?depth={1}')
    '''
        def place_orders(self,market,side,size,price,type_ord):
            return self._post(f'/orders',{   'market': market,
                                             'side': side,
                                             'price': price,
                                             'size': size,
                                             'type': type_ord,
                                         })
    '''

    
    def place_conditional_orders(self,market,side,size,type_ord,open_price,tp):
        return self._post(f'/conditional_orders',{    'market': market, 
                                                      'side': side,
                                                      'size': size,
                                                      'type': type_ord,
                                                      'triggerPrice':tp,
                                                      'orderPrice':open_price
                                                 })
    
    def historicalPrice(self,market_name,resolution,limit,start_time,end_time):
        return self._get(f'/markets/{market_name}/candles?resolution={resolution}&limit={limit}&start_time={start_time}&end_time={end_time}')
    
    def get_open_orders(self,market):
        return self._get(f'/orders?market={market}')
    
    def place_orders(self,market,side,size,price,type_ord):
        tm = time.localtime() # get struct_time
        time_string = time.strftime("%Y-%m-%d, %H:%M:%S", tm)
        tr = self.get_ticker(market)
        if(side == 'buy'):
            price = tr['asks'][0][0]
        else:
            price = tr['bids'][0][0]
            
        return {"createdAt": time_string,
                "filledSize": 0,
                "future": market,
                "id": int(time.time()),
                "market": market,
                "price": price,
                "remainingSize": size,
                "side": side,
                "size": size,
                "status": "open",
                "type": type_ord,
              }
    
    def get_open_conditional_order(self,market):
        return self._get(f'/conditional_orders?market={market}')
    
    def get_conditional_order(self,conditional_order_id):
        return self._get(f'/conditional_orders/{conditional_order_id}/triggers')